In [14]:
import torch
import os
import pandas as pd
import csv
import glob
from torch import nn
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from models import *
from dataset import *

In [5]:
if torch.cuda.is_available():
    device='cuda:0'
    print('gpu found')
else:
    device='cpu'

gpu found


In [15]:
import gc
gc.collect()

143

In [21]:
N = 100

In [90]:
def update_performance_info(curr_loss, performance):
    pass


def train_epoch(dataloader, model, criterion, optimizer, performance_info):
    for i, (X, y) in enumerate(dataloader):
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        update_performance_info(loss, performance_info)
        if i % 100 == 0:
            print(f"Training batch loss {loss}")
            
    

In [18]:
# TODO
gene = 'MKI67|4288'
patient_info_file = '../data/patients_ids.txt'
with open(patient_info_file, 'r') as f:
    patient_list = f.read().splitlines()

label_info_file = f'../data/labels/label_{gene}.csv'
with open(label_info_file, 'r') as f:
    reader = csv.reader(f)
    label_dict = {rows[0]:rows[1] for rows in reader}

feature_files = list(glob.glob('../data/features/*.npy'))
    
feature_dataset = tileFeatureDataset(feature_files, label_dict)

In [ ]:
f, l = feature_dataset[0]
f.shape

In [94]:
# Start training
batch = 8
data_loader = DataLoader(feature_dataset, batch_size=batch, shuffle=True)
# hyperparameters
model = AttnClassifier.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
epoch = 10
patients = 3

for e in range(epoch):
    for i, (X, y) in enumerate(data_loader):
        print(X.shape)
        output = model(X.to(device))
        print(output.shape)
        loss = criterion(output, y.to(device))
        print(loss)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # if i % 100 == 0:
        #     print(f"Training batch loss {loss}")

Input for patient TCGA-A8-A08X exceeds limit, truncating..
Input for patient TCGA-BH-A0HQ exceeds limit, truncating..
Input for patient TCGA-D8-A1JL exceeds limit, truncating..
Input for patient TCGA-BH-A18M exceeds limit, truncating..
Input for patient TCGA-D8-A27M exceeds limit, truncating..
Input for patient TCGA-A2-A25D exceeds limit, truncating..
Input for patient TCGA-C8-A27A exceeds limit, truncating..
Input for patient TCGA-AC-A2FO exceeds limit, truncating..
torch.Size([8, 3, 100, 224, 224])


RuntimeError: CUDA out of memory. Tried to allocate 460.00 MiB (GPU 0; 14.76 GiB total capacity; 13.52 GiB already allocated; 277.75 MiB free; 13.52 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF